# Cartograms with QGIS

Thematic maps are powerful tools to represent the spatial distributions of various phenomena. However, areal units may be misleading in certain cases. Most famously this is the case with choropleth election maps, such as the one below – large rural areas that dominate the map might indicate a landslide win. Yet, it is the populace that votes, and the number of voters per administrative unit varies wildly.


![US election map](Figures/US-election-map-2012.jpg)

*Source: Nolan, J. V. & Congressional Cartography Program. (2012) United States presidential election , results by county, November 6. \[Washington, D.C.: Congressional Cartography Program\] \[Map\] Retrieved from the Library of Congress, https://www.loc.gov/item/2012587691/*

If area =/= importance, yet we want to show something spatially, what can be done? One solution is cartograms, mixtures of maps and diagrams, which distort geographical space to represent a variable. A good cartogram will exploit the underlying geographical knowledge of the reader while telling a story about the phenomena that is being mapped.

This tutorial will introduce two methods to make area cartograms in QGIS and demonstrate how areal distortion might benefit your maps. 

In this tutorial, you will learn:
1. About the principles of cartograms.
2. When a cartogram may be the correct choice of visualization.
3. How to create two types of cartograms with QGIS.

You can find a more thorough explanation of cartograms and more examples [in this article by Kenneth Field](https://gistbok-topics.ucgis.org/CV-04-032).

## Prerequisites

Install two plugins:
- [Cartogram3](https://plugins.qgis.org/plugins/cartogram3/): Plugin for creating continous cartograms.
- [Shape tools](https://plugins.qgis.org/plugins/shapetools/): A collection of various tools for creating and modifying geodesic shapes. We will use it for scaling polygons.

## Data
We will use the example dataset provided by Cartogram3, which is the population of Austria by statistical [NUTS2 areas](https://ec.europa.eu/eurostat/web/nuts).

<div style="background-color: #66f2ff; padding: 10px; border-left: 5px solid #00a4b3; margin-bottom: 10px;">

Load the example data to QGIS from the drop-down menu *Vector* > *Cartogram* > *Add sample dataset*

</div>

The plugin applies a styling that nicely highlights why cartograms are sometimes handy – the labels are population density. The capital region in Vienna, and the associated NUTS2 region, is in a wholly another level of population density that the rest of the country.

<div style="background-color: #39f98f; padding: 10px; border-left: 5px solid #059445; margin-bottom: 10px;">

No style file for this map; it's using the style the Cartogram plugin applies by default.
    
</div>

![Cartogram initial state](Figures/Cartograms-austria-raw.jpeg)

## Non-contigous cartogram
The simplest approach to not let geography dominate our map? Just scale the geometries according to a value of our choice. In this example, we'll use population.

We'll follow the method laid out in [Olson (1976): Non-contigous cartograms](https://doi.org/10.1111/j.0033-0124.1976.00371.x) and scale the geometries to be X times smaller and larger than initially.

To do that, we'll need a scale factor, which is:

$$
\text{Scale factor} = k \times \sqrt{\frac{\text{value}}{\text{area}}}
$$

Where:

- Value: e.g. population 
- Area: e.g. km²
- k: 1 / the density of an "anchor unit". The anchor unit will stay the same size while all others are scaled relative to it. It's wise to choose an anchor unit that has a high density, otherwise some areas will be scaled up and start to overlap. We'll use *Vienna* as the anchor unit.

Let's apply that formula and calculate a scale factor for our data:

<div style="background-color: #66f2ff; padding: 10px; border-left: 5px solid #00a4b3; margin-bottom: 10px;">

1. Add a new fields with **Field calculator**.
2. First one population density squared:
    - Output field name: *pop_density_sqrt*
    - Output field type: *Decimal*
    - Expression: `sqrt( "pop20170101" /  "area_km2" )`
3. Then use that to calculate the scaling factor:
    - Output field name: *scaling_factor*
    - Output field type: *Decimal*
    - Expression: `1 / maximum( "pop_density_sqrt") * "pop_density_sqrt"`
        - Note: We use `maximum( "pop_density_sqrt")`as a simple way to get the highest value (the one for Vienna, our anchor unit) from the field.
</div>

With that, we have a field that stores the scaling values. For Vienna, that value is 1 and for the others, less than 1.

Let's apply scaling:

<div style="background-color: #66f2ff; padding: 10px; border-left: 5px solid #00a4b3; margin-bottom: 10px;">

1. Run the processing tool *Geodesic transformations* (part of the Shape tools plugin: be sure to install it!)
    - Apply a data-driven override for *Scale factor about the centroid*
    - Expression `"scaling_factor"`
        - In other words, the expression will return the value in this field.
</div>

![Cartogram non-contiguous](Figures/Cartograms-austria-non-contiguous.jpeg)

NB! The regions are styled simply by their id. In a thematic map, the regions would be colored by, for example, the most popular political party.

While this map keeps the familiar shape of the regions, the areas can get quite miniscule. In addition, it's just not very visually striking and needs the actual map of Austria in the background to imply which areas are which.

## Contigous cartogram
What if we want to keep the topology of our maps – e.g., the areas should have the same neighbours? This will mean sacrificing the shape of our regions, but it should produce a more interesting and accurate cartogram. Let's try it out on our data using the algorithm implemented in the cartogram3 plugin.

<div style="background-color: #66f2ff; padding: 10px; border-left: 5px solid #00a4b3; margin-bottom: 10px;">

    
Run **Compute cartogram** (*Vector* > *Cartogram* > *Compute cartogram*). 
- Field: *pop20170101*
    
- You can lower max average error for a more accurate result, or keep it intact.

</div>

Notice how the shapes of the cartogram are distorted – especially around the capital region. With this few areas, even most of the shapes are relatively well preserved, though that is dependent on the data that you're working on. How many units and how much the value varies between them affects the distortion quite significantly.

![Cartogram contiguous](Figures/Cartograms-austria-contiguous.jpeg)

## What are cartograms, really?

Let's still consider what we are actually representing above in more detail. Calculate two fields to the contiguous cartogram that you just created.

<div style="background-color: #66f2ff; padding: 10px; border-left: 5px solid #00a4b3; margin-bottom: 10px;">

    
Use Field calculator to add a fields about how population and distorted land area are distributed between the NUTS2 areas.
    
1. Population distribution:
    1. Name: `pop_proportion`
    2. Type: Decimal
    3. Expression: `"pop20170101" / sum( "pop20170101" ) * 100`
2. Area distribution:
    1. Name: `distorted_area_proportion`
    2. Type: Decimal
    3. Expression: `$area / sum( $area ) * 100`

</div>

You will have values similar to these, depending a bit on the parameters used for creating the cartogram. A difference column has been added for clarity:

| nuts2 | distorted_area_proportion (%) | pop_proportion (%) | difference |
|-------|---------------------------|----------------|------------|
| AT13  | 19.29                     | 21.29          | -2         |
| AT12  | 19.162                    | 18.986         | 0.176      |
| AT31  | 16.44                     | 16.7           | -0.260     |
| AT22  | 14.24                     | 14.103         | 0.137      |
| AT33  | 9.83                      | 8.505          | 1.325      |
| AT21  | 6.864                     | 6.395          | 0.469      |
| AT32  | 6.138                     | 6.262          | -0.124     |
| AT34  | 4.343                     | 4.43           | -0.087     |
| AT11  | 3.694                     | 3.328          | 0.366      |


Notice that the proportion of population and proportion of land area after distortion are only different by a few percentage points at most.

Thinking of the *diagram* part of cartograms, what are they then? 

"**Cartograms are best understood as pie charts where the segment of the pie is reshaped to look like a country.**" ([Worldmapper.org FAQ](https://worldmapper.org/faq/#3), quoted in [cartogram3 readme](https://github.com/austromorph/cartogram3)).

Thinking of cartograms as pie charts helps to consider when they are useful and when not. For example, the dataset may have many values with a majority of them being very small – a pie chart visualization quickly becomes unreadable. 


Consider: would a cartogram of 309 municipalities of Finland by population be useful and if so, when? ([Data from Statistics Finland](https://stat.fi/org/avoindata/paikkatietoaineistot/vaesto_tilastointialueittain_en.html))

![Finland contiguous](Figures/Cartograms-Finland.jpeg)

Going beyond the two cartogram types introduced here, you may be interested in [graphical or gridded cartograms](https://gistbok-topics.ucgis.org/CV-04-032). However, we are not aware of ready tools to produce them in QGIS, and it making them may require some manual work or looking for tools beyond QGIS.

## Replicating the processing flow of this notebook

To replicate this processing flow, run the processing model *cartograms-model.model3*. Open the model in QGIS from the leftmost button below *Processing toolbox* -> *Open existing model*. You will need to add the example data to the project before running the model. Please also note that this model includes some hard-coded field-names (as do most of these models!). They are meant for replicating this notebook, and repurposing them for general use might require some modifications.

![Cartogram model](Figures/cartograms-model.png)